In [1]:
import pandas as pd
import numpy as np
import re
import folium
from folium import plugins
import geocoder
import colorbrewer

In [2]:
# Escala para la ciudad de México
# Delimitar la base de datos únicamente a alcaldías de la CDMX e identificar número de frecuencia por delitos

In [3]:
datos_fgj = pd.read_csv('PGJ.csv')

In [4]:
datos_fgj.head(5)

,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,alcaldia_hechos,colonia_hechos,ao_inicio,mes_inicio,fecha_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint
0,2017.0,Octubre,2017-10-06 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN AZCAPOTZALCO,AZ-1,UI-3SD,IZTAPALAPA,APATLACO,2017,Noviembre,2017-11-15 20:14:33,REAL DEL MONTE,NaN,-99.116256,19.380585,"19.3805845107,-99.1162559436"
1,2017.0,Noviembre,2017-11-15 20:00:00,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,INVESTIGACIÓN EN TLALPAN,TLP-2,UI-3CD,TLALPAN,MIGUEL HIDALGO 2A SECCIÓN,2017,Noviembre,2017-11-15 20:15:06,JESUS LECUONA,OAXACA,-99.193514,19.286033,"19.2860327788,-99.1935140733"
2,2017.0,Noviembre,2017-11-15 18:30:00,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN IZTAPALAPA,IZP-4,UI-3SD,IZTAPALAPA,DR. ALFONSO ORTIZ TIRADO,2017,Noviembre,2017-11-15 20:18:33,SAN RAFAEL ATLIXCO,NaN,-99.073700,19.387269,"19.3872690138,-99.0736999107"
3,2017.0,Noviembre,2017-11-15 17:32:00,ROBO DE MOTOCICLETA CON VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,INVESTIGACIÓN EN MIGUEL HIDALGO,MH-2,UI-1SD,MIGUEL HIDALGO,LOMAS DE SOTELO I,2017,Noviembre,2017-11-15 20:19:27,AV. INGENIEROS MILITARES,NaN,-99.217985,19.453354,"19.4533537449,-99.2179851004"
4,2017.0,Noviembre,2017-11-14 19:00:00,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN IZTAPALAPA,IZP-9,UI-3SD,IZTAPALAPA,2A. AMPLIACIÓN SANTIAGO ACAHUALTEPEC,2017,Noviembre,2017-11-15 20:20:20,EVA SAMANO,LEONA VICARIO,-99.012944,19.347705,"19.3477045484,-99.012944086"


In [5]:
#Filtro la base únicamente para datos cdmx
PGJ = datos_fgj.replace(np.nan, '', regex=True)

In [6]:
new = PGJ[['ao_hechos', 'mes_hechos', 'delito', 'categoria_delito', 'alcaldia_hechos', 'colonia_hechos']]
alcaldias_cdmx = ['ALVARO OBREGON','BENITO JUAREZ','AZCAPOTZALCO', 'COYOACAN', 'CUAJIMALPA', 'CUAUHTEMOC', 'GUSTAVO A. MADERO', 'IZTACALCO', 'IZTAPALAPA', 'MAGDALENA CONTRERAS', 'MIGUEL HIDALGO', 'MILPA ALTA', 'TLAHUAC', 'TLALPAN', 'VENUSTIANO CARRANZA', 'XOCHIMILCO']

In [7]:
contain_cdmx = new[new['alcaldia_hechos'].str.contains('ALVARO|BENITO|AZCAPOT|COYOA|CUAJI|CUAUHTE|GUSTAVO A.|IZTACA|IZTAPA|MAGDALENA|MIGUEL HID|MILPA|TLAHUAC|TLALPAN|VENUSTIANO|XOCHIMIL')]
contain_cdmx

,ao_hechos,mes_hechos,delito,categoria_delito,alcaldia_hechos,colonia_hechos
0,2017,Octubre,FRAUDE,DELITO DE BAJO IMPACTO,IZTAPALAPA,APATLACO
1,2017,Noviembre,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,TLALPAN,MIGUEL HIDALGO 2A SECCIÓN
2,2017,Noviembre,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,IZTAPALAPA,DR. ALFONSO ORTIZ TIRADO
3,2017,Noviembre,ROBO DE MOTOCICLETA CON VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,MIGUEL HIDALGO,LOMAS DE SOTELO I
4,2017,Noviembre,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,IZTAPALAPA,2A. AMPLIACIÓN SANTIAGO ACAHUALTEPEC
...,...,...,...,...,...,...
1101094,2020,Octubre,LESIONES INTENCIONALES POR ARMA BLANCA,DELITO DE BAJO IMPACTO,IZTACALCO,JUVENTINO ROSAS
1101095,2020,Octubre,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,XOCHIMILCO,SAN MATEO XALPA
1101096,2020,Octubre,ABUSO SEXUAL,DELITO DE BAJO IMPACTO,GUSTAVO A MADERO,7 DE NOVIEMBRE
1101097,2020,Octubre,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,TLALPAN,SAN PEDRO MÁRTIR


In [8]:
delitos_cdmx = contain_cdmx.groupby(['alcaldia_hechos','delito']).size().reset_index()
delitos_cdmx

,alcaldia_hechos,delito,0
0,ACUAMANALA DE MIGUEL HIDALGO,TRATA DE PERSONAS,1
1,ALVARO OBREGON,ABANDONO DE PERSONA,141
2,ALVARO OBREGON,ABORTO,65
3,ALVARO OBREGON,ABUSO DE AUTORIDAD,155
4,ALVARO OBREGON,ABUSO DE AUTORIDAD Y USO ILEGAL DE LA FUERZA P...,220
...,...,...,...
4253,XOCHIMILCO,VIOLACION EQUIPARADA POR CONOCIDO,2
4254,XOCHIMILCO,VIOLACION EQUIPARADA Y ROBO DE VEHICULO,1
4255,XOCHIMILCO,VIOLACION TUMULTUARIA,6
4256,XOCHIMILCO,VIOLACION TUMULTUARIA EQUIPARADA POR CONOCIDO,1


In [9]:
#delitos_cdmx.to_csv('Delitos CDMX.csv')

In [10]:
feminicidio = datos_fgj[datos_fgj['delito'].str.contains('FEMINICIDIO')]
feminicidio.replace(np.nan, '', regex=True)
feminicidio.head(3)

,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,alcaldia_hechos,colonia_hechos,ao_inicio,mes_inicio,fecha_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint
11302,2017.0,Agosto,2017-08-17 22:30:00,FEMINICIDIO,HOMICIDIO DOLOSO,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-4,UI-3CD,GUSTAVO A MADERO,15 DE AGOSTO,2017,Agosto,2017-08-18 01:55:34,GUADALUPE VICTORIA NUMERO 60 INTERIOR 3,--,-99.107190,19.484555,"19.4845550683,-99.1071904777"
13022,2017.0,Marzo,2017-03-27 17:25:00,FEMINICIDIO,HOMICIDIO DOLOSO,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE H...,FCIH,3 CON DETENIDO 3 C/D,MERIDA,NaN,2017,Agosto,2017-08-29 20:42:14,CALLE 28,"YUCATAN, ESTADO DE MEXICO",NaN,NaN,NaN
31913,2016.0,Julio,2016-07-12 03:30:00,FEMINICIDIO,HOMICIDIO DOLOSO,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-4,UI-3CD,GUSTAVO A MADERO,AMPLIACIÓN GABRIEL HERNÁNDEZ,2016,Julio,2016-07-12 04:05:04,M. FLORES,EVA SAMANO,-99.099265,19.504712,"19.50471156,-99.0992649081"


In [11]:
#feminicidio.to_csv('Feminicidio_CDMX.csv') 
# 278 casos

In [12]:
años = feminicidio.groupby(by="ao_hechos").size().reset_index()
años

,ao_hechos,0
0,2000.0,1
1,2015.0,1
2,2016.0,48
3,2017.0,47
4,2018.0,42
5,2019.0,73
6,2020.0,65


In [13]:
#años.to_csv('feminicidio_año.csv')

In [14]:
alcaldias = feminicidio.groupby(['alcaldia_hechos','colonia_hechos']).size().reset_index()
alcaldias

,alcaldia_hechos,colonia_hechos,0
0,ALVARO OBREGON,2A. AMPLIACION JALALPA EL GRANDE,1
1,ALVARO OBREGON,BALCONES DE CEHUAYO,1
2,ALVARO OBREGON,CARLOS A. MADRAZO,1
3,ALVARO OBREGON,COVE,1
4,ALVARO OBREGON,JALALPA EL GRANDE 1ER. REACOMODO,1
...,...,...,...
195,XOCHIMILCO,SAN MATEO XALPA,2
196,XOCHIMILCO,SANTA CECILIA TEPETLAPA,2
197,XOCHIMILCO,SANTA CRUZ ACALPIXCA - PUEBLO,1
198,XOCHIMILCO,SANTIAGO TEPALCATLALPAN,1


In [15]:
coordenadas = feminicidio[['alcaldia_hechos','longitud','latitud']]
coordenadas.head(5)
len(coordenadas)
new = coordenadas.dropna()

In [16]:
#Creamos el mapa
mapa_feminicidio = folium.Map(tiles = 'stamentoner',location=[19.4978, -99.1269],zoom_start=11)

#crear los marcadores con iterrows

for indice, fila in new.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red")
      
  ).add_to(mapa_feminicidio)

In [17]:
new.head(3)

,alcaldia_hechos,longitud,latitud
11302,GUSTAVO A MADERO,-99.107190,19.484555
31913,GUSTAVO A MADERO,-99.099265,19.504712
34222,ALVARO OBREGON,-99.252319,19.360315


In [18]:
mapa_feminicidio

In [19]:
alcaldias_feminicidio = feminicidio.groupby(['alcaldia_hechos']).size().reset_index()
alcaldias_feminicidio.head(2)

,alcaldia_hechos,0
0,ALVARO OBREGON,16
1,AZCAPOTZALCO,9


In [20]:
#Prueba CHOROPLETH

#choro_map = folium.Map(location=[19.4978, -99.1269], zoom_start= 11)

#world_geo = r'alcaldias.geojson' #geojson file

#create a plain world map
#world_map = folium.Map(location=[0, 0], zoom_start=2, tiles= 'stamentoner')

#folium.Choropleth(geo_data = mx_prueba, data = alcaldias_feminicidio, columns= ['alcaldia_hechos', '0'], key_on=feature.properties )

In [21]:
#Create map

#world_map.choropleth(
#    geo_data = world_geo,
#    data= feminicidio,
#    columns=['alcaldia_hechos','delito'],
#    key_on='feature.properties.name',
#    threshold_scale = [0,100,500,1000],
#    fill_color = 'YlGn',
#    fill_opacity= 0.7,
#    line_opacity = 0.2,
#    legend_name = 'Feminicidio CDMX',
#   reset= True
#)

In [22]:
#PRUEBA BUBBLE MAP

cdmx = folium.Map(
    tiles = 'stamentoner',
    location=[19.4978, -99.1269],
    zoom_start=11
)

#circulo con radio en metros para iztapalapa 
folium.Circle(
    radius=1900,
    location=[19.35, -99.05],
    popup="Iztapalapa",
    color="crimson",
    fill=False
).add_to(cdmx)

#Gustavo A Madero
folium.Circle(
    radius=1900,
    location=[19.482, -99.09100],
    popup="Gustavo A Madero",
    color="crimson",
    fill=False
).add_to(cdmx)

#Tlalpan
folium.Circle(
    radius=1900,
    location=[19.2875, -99.1675],
    popup="Tlalpan",
    color="crimson",
    fill=False
).add_to(cdmx)

#Milpa Alta
#folium.Circle(
 #   radius=1800,
  #  location=[19.19251, -99.02317],
   # popup="Benito Juárez",
  #  color="Blue",
   # fill=False
#).add_to(cdmx)

#Cuajimalpa
#folium.Circle(
 #   radius=1800,
  #  location=[19.3692, -99.29089],
   # popup="Benito Juárez",
    #color="Blue",
    #fill=False
#).add_to(cdmx)

#Magdalena Contreras
#folium.Circle(
 #   radius=1800,
 #   location=[19.33212, -99.21118],
#  popup="Benito Juárez",
#    color="Blue",
 #   fill=False
#).add_to(cdmx)

#visualizamos
cdmx

In [23]:
#IZTAPALAPA
iztapalapa = new[new['alcaldia_hechos']== 'IZTAPALAPA']
iztapalapa.head()

,alcaldia_hechos,longitud,latitud
36579,IZTAPALAPA,-99.026563,19.379476
59924,IZTAPALAPA,-99.099530,19.348891
73267,IZTAPALAPA,-99.039040,19.325444
147773,IZTAPALAPA,-99.103992,19.326284
176241,IZTAPALAPA,-99.018191,19.328273


In [24]:
#Mapa_Iztapalapa 
mapa_iztapalapa = folium.Map(tiles = 'stamentoner',location=[ 19.35529, -99.06224],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in iztapalapa.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_iztapalapa)

In [25]:
mapa_iztapalapa

In [26]:
col_izt = feminicidio[feminicidio['alcaldia_hechos']== 'IZTAPALAPA']
colonias_izt = col_izt[['colonia_hechos']]
len(colonias_izt)

50

In [48]:
colonias_izt.sort_values(by=['colonia_hechos'])
#Zonas de peligro: Consejo Agrarista Mexicano, El paraíso, Los Reyes Culhuacán, Reforma Política, San Antonio, San Juan Xalpa, Tulyehualco

,colonia_hechos
420551,2A. AMPLIACIÓN SANTIAGO ACAHUALTEPEC
960558,ACULCO
782198,AMPLIACIÓN EMILIANO ZAPATA
774352,APATLACO
394222,CENTRAL DE ABASTO
914366,CHINAMPAC DE JUÁREZ
522421,CONSEJO AGRARISTA MEXICANO
865420,CONSEJO AGRARISTA MEXICANO
73267,DESARROLLO URBANO QUETZALCOATL
821473,EJERCITO DE ORIENTE ZONA PEÑON


In [28]:
#Gustavo A. Madero
GustavoAM= new[new['alcaldia_hechos']== 'GUSTAVO A MADERO']
GustavoAM.head()

,alcaldia_hechos,longitud,latitud
11302,GUSTAVO A MADERO,-99.107190,19.484555
31913,GUSTAVO A MADERO,-99.099265,19.504712
34660,GUSTAVO A MADERO,-99.147674,19.526110
38692,GUSTAVO A MADERO,-99.134741,19.522023
47911,GUSTAVO A MADERO,-99.135128,19.570027


In [29]:
#Mapa_Gustavo A. Madero
mapa_GustavoAM = folium.Map(tiles = 'stamentoner',location=[19.49392, -99.11075],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in GustavoAM.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_GustavoAM)

In [30]:
mapa_GustavoAM

In [31]:
col_gustavo = feminicidio[feminicidio['alcaldia_hechos']== 'GUSTAVO A MADERO']
colonias_gustavo = col_gustavo[['colonia_hechos']]
len(colonias_gustavo)
#colonias_gustavo.sort_values(by=['colonia_hechos'])
#zonas de peligro: Campestre Aragón, Cuchilla del Tesoro, Loma la Palma, Malacates y Vallejo

34

In [32]:
#Cuauhtemoc
Cuauhtemoc= new[new['alcaldia_hechos']== 'CUAUHTEMOC']
Cuauhtemoc.head()

,alcaldia_hechos,longitud,latitud
123107,CUAUHTEMOC,-99.146084,19.419870
243566,CUAUHTEMOC,-99.126300,19.435005
395087,CUAUHTEMOC,-99.134360,19.418080
436970,CUAUHTEMOC,-99.131071,19.449469
471053,CUAUHTEMOC,-99.130454,19.440086


In [33]:
mapa_Cuauhtemoc = folium.Map(tiles = 'stamentoner',location=[19.3086 , -99.2249],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in Cuauhtemoc.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_Cuauhtemoc)

In [34]:
mapa_Cuauhtemoc

In [35]:
col_Cuauhtemoc = feminicidio[feminicidio['alcaldia_hechos']== 'CUAUHTEMOC']
colonias_Cuau = col_Cuauhtemoc[['colonia_hechos']]
len(colonias_Cuau)
#colonias_Cuau.sort_values(by=['colonia_hechos'])
#CENTRO (7), Guerrero, Obrera

23

In [36]:
#Alvaro Obregon
AlvObr= new[new['alcaldia_hechos']== 'ALVARO OBREGON']
AlvObr.head()

,alcaldia_hechos,longitud,latitud
34222,ALVARO OBREGON,-99.252319,19.360315
42028,ALVARO OBREGON,-99.235187,19.334152
161122,ALVARO OBREGON,-99.200128,19.367548
185150,ALVARO OBREGON,-99.239176,19.370306
365713,ALVARO OBREGON,-99.248020,19.331010


In [37]:
mapa_AlvObr = folium.Map(tiles = 'stamentoner',location=[19.3086 , -99.2249],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in AlvObr.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_AlvObr)

In [38]:
mapa_AlvObr

In [39]:
col_AlvObr = feminicidio[feminicidio['alcaldia_hechos']== 'ALVARO OBREGON']
colonias_AlvObr = col_AlvObr[['colonia_hechos']]
colonias_AlvObr.sort_values(by=['colonia_hechos'])
len(colonias_AlvObr) 
#16 casos de los cuales ninguno registra una colonia repetida

16

In [40]:
#Tlalpan 
Tlalpan= new[new['alcaldia_hechos']== 'TLALPAN']
Tlalpan.head()

,alcaldia_hechos,longitud,latitud
146195,TLALPAN,-99.205679,19.270350
164767,TLALPAN,-99.215065,19.229772
187085,TLALPAN,-99.142280,19.189630
216437,TLALPAN,-99.176472,19.233653
245185,TLALPAN,-99.158268,19.253563


In [41]:
#Mapa_Tlalpan
mapa_Tlalpan = folium.Map(tiles = 'stamentoner',location=[19.3086 , -99.2249],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in Tlalpan.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_Tlalpan)

In [42]:
mapa_Tlalpan

In [49]:
col_tlalpan = feminicidio[feminicidio['alcaldia_hechos']== 'TLALPAN']
colonias_tlalpan = col_tlalpan[['colonia_hechos']]
len(colonias_tlalpan)
colonias_tlalpan.sort_values(by=['colonia_hechos'])
#zonas de peligro: Magdalena Petlacalco, San Andrés Totoltepec, San Lorenzo Huipulco, SAN MIGUEL AJUSCO (6), 
#SAN MIGUEL TOPILEJO, SAN PEDRO MARTIR, SANTO TOMÁS AJUSCO (6) - 
#OJO AQUÍ: PATRONES DE COLONIAS SÚPER MARCADOS 

,colonia_hechos
912421,HÉROES DE PADIERNA
146195,JARDINES DEL AJUSCO
216437,LA MAGDALENA PETLACALCO
340796,LA MAGDALENA PETLACALCO
929790,MARÍA ESTHER ZUNO DE ECHEVERRÍA
581228,MESA DE LOS HORNOS
886807,MIGUEL HIDALGO 4A SECCIÓN
783878,NARCISO MENDOZA SUPER MANZANA 6 VILLA COAPA
521139,PRADO COAPA 2A SECCIÓN
536758,RESIDENCIAL MIRAMONTES


In [44]:
#Xochimilco
Xochimilco= new[new['alcaldia_hechos']== 'XOCHIMILCO']
Xochimilco.head()

,alcaldia_hechos,longitud,latitud
38634,XOCHIMILCO,-99.091641,19.252803
41683,XOCHIMILCO,-99.016346,19.254370
48205,XOCHIMILCO,-99.111734,19.270672
160820,XOCHIMILCO,-99.111667,19.270649
307776,XOCHIMILCO,-99.095905,19.207995


In [45]:
#Mapa Xochimilco
mapa_Xoch = folium.Map(tiles = 'stamentoner',location=[19.3086 , -99.2249],zoom_start=12.5)

#crear los marcadores con iterrows

for indice, fila in Xochimilco.iterrows():
  folium.Marker(
      location=[fila.loc['latitud'], fila.loc['longitud']],
      icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa")
  ).add_to(mapa_Xoch)

In [46]:
mapa_Xoch

In [47]:
col_Xoch = feminicidio[feminicidio['alcaldia_hechos']== 'XOCHIMILCO']
colonias_Xoch = col_Xoch[['colonia_hechos']]
len(colonias_Xoch)
colonias_Xoch.sort_values(by=['colonia_hechos'])

,colonia_hechos
1001040,BARRIO 18
939949,BARRIO 18
654677,BARRIO XALTOCAN
38634,CALTONGO
1029999,CALTONGO
475733,CALTONGO
896314,LA JOYA
802863,PARQUE ECOLOGICO DE XOCHIMILCO
523529,PARQUE ECOLOGICO DE XOCHIMILCO
1077750,SAN GREGORIO ATLAPULCO
